In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Stage 2: Building MVP: - 01 Data Processing


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/workshops/rag-ops/2.1_mvp_data_processing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fworkshops%2Frag-ops%2F2.1_mvp_data_processing.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/workshops/rag-ops/2.1_mvp_data_processing.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/workshops/rag-ops/2.1_mvp_data_processing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

This notebook is the first in a series designed to guide you through building a Minimum Viable Product (MVP) for a Multimodal Retrieval Augmented Generation (RAG) system using the Gemini API in Vertex AI.

**Here's what you'll achieve:**

* **Process diverse data:**  Extract information from PDFs, audio files, and video files. This includes text extraction from PDFs, and generating summaries and analyses from audio and video content.
* **Build from scratch:** Understand the core concepts of data processing for RAG without relying on third-party libraries like LangChain or LlamaIndex. This hands-on approach allows you to learn the fundamentals and customize your solution later.
* **Lay the foundation:** This notebook sets the stage for subsequent notebooks that cover data chunking, embeddings, retrieval, generation, and evaluation.

By the end of this series, you'll have a working MVP and the knowledge to further develop and optimize your own multimodal RAG system.


## Getting Started

### Install Vertex AI SDK for Python


In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform PyPDF2

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

In [ ]:
import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information, GCS Bucket and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
import os
import sys

from google.cloud import storage
import vertexai

PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"
BUCKET_NAME = "mlops-for-genai"

if PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

if not PROJECT_ID or PROJECT_ID == "[your-project-id]" or PROJECT_ID == "None":
    raise ValueError("Please set your PROJECT_ID")


vertexai.init(project=PROJECT_ID, location=LOCATION)

# Initialize cloud storage
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)

In [ ]:
# # Variables for data location. Do not change.

PRODUCTION_DATA = "multimodal-finanace-qa/data/unstructured/production/"
PICKLE_FILE_NAME = "data_extraction_dataframe.pkl"

### Import libraries


In [ ]:
import asyncio
import asyncio.locks

# Library
from io import BytesIO
import os
import pickle
import time

import PyPDF2
from google.cloud import storage
import pandas as pd
import psutil
from rich import print as rich_print
from rich.markdown import Markdown as rich_Markdown
from tenacity import retry, stop_after_attempt, wait_random_exponential
from vertexai.generative_models import (
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

### Load the Gemini 1.5 models

To learn more about all [Gemini API models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).

The Gemini model family has several model versions. You will start by using Gemini 1.5 Flash. Gemini 1.5 Flash is a more lightweight, fast, and cost-efficient model. This makes it a great option for prototyping.


In [ ]:
MODEL_ID_FLASH = "gemini-1.5-flash-002"  # @param {type:"string"}
MODEL_ID_PRO = "gemini-1.5-pro-002"  # @param {type:"string"}


gemini_15_flash = GenerativeModel(MODEL_ID_FLASH)
gemini_15_pro = GenerativeModel(MODEL_ID_PRO)

In [ ]:
# @title Gemini API Call Functions


def get_gemini_response(
    model,
    generation_config=None,
    safety_settings=None,
    uri_path=None,
    mime_type=None,
    prompt=None,
):
    if not generation_config:
        generation_config = {
            "max_output_tokens": 8192,
            "temperature": 1,
            "top_p": 0.95,
        }

    if not safety_settings:
        safety_settings = {
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        }

    uri = "gs://" + uri_path
    file = Part.from_uri(mime_type=mime_type, uri=uri)
    responses = model.generate_content(
        [file, prompt],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )
    final_response = []
    for response in responses:
        try:
            final_response.append(response.text)
        except ValueError:
            # print("Something is blocked...")
            final_response.append("blocked")

    return "".join(final_response)


def get_load_dataframes_from_gcs():
    gcs_path = "multimodal-finanace-qa/data/structured/" + PICKLE_FILE_NAME
    # print("GCS PAth: ", gcs_path)
    blob = bucket.blob(gcs_path)

    # Download the pickle file from GCS
    blob.download_to_filename(f"{PICKLE_FILE_NAME}")

    # Load the pickle file into a list of dataframes
    with open(f"{PICKLE_FILE_NAME}", "rb") as f:
        dataframes = pickle.load(f)

    # Assign the dataframes to variables
    extracted_text, audio_metadata_flash, video_metadata_flash = dataframes

    return extracted_text, audio_metadata_flash, video_metadata_flash

![](https://storage.googleapis.com/mlops-for-genai/multimodal-finanace-qa/img/building_index_batch_processing_flow_animated.svg)

## Step 1: Data Processing

![](https://storage.googleapis.com/mlops-for-genai/multimodal-finanace-qa/img/data_processing_step1.png)

In [ ]:
# # [Optional]
# # You can load the dataframes from GCS which are pre-indexed to save time and cost
# #  Uncomment the code below

# extracted_text, audio_metadata_flash, video_metadata_flash = get_load_dataframes_from_gcs()

Or else you can continue to run the cells. Do note that it will take time to run through all the files.

### Text Extraction

![](https://storage.googleapis.com/gemini-lavi-asset/img/Step1.png)

In [ ]:
# @title Helper Functions - Text Extraction from PDF


def get_text_from_pdf(bucket):
    extracted_text = []
    # Iterate over all blobs (files) in the bucket
    for blob in bucket.list_blobs():
        if blob.name.startswith(PRODUCTION_DATA):
            if blob.name.lower().endswith(".pdf"):  # Check if the file is a PDF
                # Download the PDF to a BytesIO object
                pdf_content = BytesIO(blob.download_as_bytes())
                try:
                    # Process the PDF using PyPDF2
                    pdf_reader = PyPDF2.PdfReader(pdf_content)
                    text = ""
                    pdf_data = []
                    text_type = "/".join(blob.name.split("/")[1:-1])
                    filename = blob.name.split("/")[-1]
                    for page_num in range(len(pdf_reader.pages)):
                        page = pdf_reader.pages[page_num]
                        text = page.extract_text()
                        if text:
                            pdf_data.append(
                                {
                                    "text_type": text_type,
                                    "gcs_path": "gs://"
                                    + blob.bucket.name
                                    + "/"
                                    + blob.name,
                                    "page_number": page_num + 1,
                                    "text": text,
                                }
                            )
                    extracted_text.extend(pdf_data)
                    # break
                except:
                    print(
                        f"Warning: Could not read PDF file '{blob.name}' (might be encrypted or corrupted)"
                    )
    return pd.DataFrame(extracted_text)

In [ ]:
%%time
extracted_text = get_text_from_pdf(bucket)

In [ ]:
extracted_text.head()

In [ ]:
file_count = pd.DataFrame(
    [
        [
            each.split("/")[-1]
            for each in extracted_text["gcs_path"].value_counts().index
        ],
        extracted_text["gcs_path"].value_counts().values,
    ]
).T
file_count.columns = ["filename", "count"]

In [ ]:
file_count.head()

In [ ]:
print("Total pages to process...", file_count["count"].sum())
print("Total files to process...", file_count.shape[0])

In [ ]:
print("Document name: ", file_count.iloc[0]["filename"])
print("Number of pages: ", file_count.iloc[0]["count"])

In [ ]:
index = 100
rich_print(
    "Name of the Document: ************\n",
    extracted_text.iloc[index]["gcs_path"].split("/")[-1],
)
rich_print("\n\nPage Number: ************\n", extracted_text.iloc[index]["page_number"])
rich_print("\n\nText: ************\n", extracted_text.iloc[index]["text"])

### Audio Summary Extraction

![](https://storage.googleapis.com/gemini-lavi-asset/img/Step1-Audio.png)

In [ ]:
# @title Audio Summary Helper Functions


def get_text_from_audio(bucket, model, prompt, time_sleep=5):
    # Iterate over all blobs (files) in the bucket
    audio_metadata = []
    for blob in bucket.list_blobs():
        if blob.name.startswith(PRODUCTION_DATA):
            if blob.name.lower().endswith(".mp3"):
                print("processing....", blob.name)
                video_type = "/".join(blob.name.split("/")[1:-1])
                gcs_path = "/".join(blob.id.split("/")[:-1])
                # print(gcs_path)
                try:
                    audio_description = get_gemini_response(
                        uri_path=gcs_path,
                        model=model,
                        mime_type="audio/mpeg",
                        prompt=prompt,
                    )
                    if audio_description:
                        audio_metadata.append(
                            {
                                "audio_gcs": "gs://"
                                + blob.bucket.name
                                + "/"
                                + blob.name,
                                "audio_type": video_type,
                                "audio_description": audio_description,
                            }
                        )
                except:
                    print("Something Failed........")
                    audio_metadata.append(
                        {
                            "audio_gcs": blob.name,
                            "audio_type": video_type,
                            "audio_description": "",
                        }
                    )
                # print("sleeping......")
                time.sleep(time_sleep)
                # break
    return pd.DataFrame(audio_metadata)


def get_gcs_uri_list(bucket, data, file_extension):
    gcs_uri_list = []
    for blob in bucket.list_blobs():
        if blob.name.startswith(data):
            if blob.name.lower().endswith(file_extension):
                gcs_path = "gs://" + "/".join(blob.id.split("/")[:-1])
                gcs_uri_list.append(gcs_path)
    return gcs_uri_list


@retry(wait=wait_random_exponential(multiplier=1, max=120), stop=stop_after_attempt(4))
async def async_generate(prompt, model, gcs_uri, mime_type):
    try:
        safety_settings = {
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        }
        # model = GenerativeModel(
        #     "gemini-1.5-flash-001",
        #     safety_settings = safety_settings
        # )
        # print("Hitting")

        response = await model.generate_content_async(
            [prompt, Part.from_uri(gcs_uri, mime_type=mime_type)],
            stream=False,
        )
        # print(len(response.text))
        return response.text
    except Exception as e:
        print("Something failed, retrying")
        print(e)
        with retry.stop_after_attempt(2) as retry_state:
            if retry_state.attempt > 2:
                return None
        raise  # Re-raise the exception for tenacity to handle


async def batch_and_profile(
    gcs_uris, prompt, model, mime_type, batch_size=2, max_concurrent=4
):
    start_time = time.time()
    memory_usage = psutil.Process().memory_info().rss / 1024**2

    semaphore = asyncio.locks.Semaphore(max_concurrent)

    async def process_batch(batch):
        async with semaphore:
            return await asyncio.gather(
                *[async_generate(prompt, model, f, mime_type) for f in batch]
            )

    batches = [
        gcs_uris[i : i + batch_size] for i in range(0, len(gcs_uris), batch_size)
    ]
    get_responses = [asyncio.create_task(process_batch(batch)) for batch in batches]
    final_response_list = [
        item for sublist in await asyncio.gather(*get_responses) for item in sublist
    ]

    end_time = time.time()
    elapsed_time = end_time - start_time
    final_memory_usage = psutil.Process().memory_info().rss / 1024**2

    print(f"Batch size: {batch_size}")
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    print(f"Initial memory usage: {memory_usage:.2f} MB")
    print(f"Final memory usage: {final_memory_usage:.2f} MB")

    return final_response_list

In [ ]:
audio_description_extraction_prompt = """Transcribe and analyze the audio, identifying key topic shifts or changes in focus. Divide the audio into segments based on these transitions.
For each segment:
* **Summarize:** Briefly describe the main topic or theme of the segment.
* **Contextualize:** Explain how this topic fits into the broader conversation or narrative.
* **Analyze:** Explore the significance of this topic, the perspectives presented, and any potential biases or underlying assumptions.
* **Synthesize:** Connect this topic to other themes or ideas mentioned in the audio, highlighting relationships and overarching patterns.
Conclude with a thematic analysis of the entire audio. Identify the most prominent themes, how they are interconnected, and the overall message or purpose of the audio.
"""

In [ ]:
%%time
audio_metadata_flash = get_text_from_audio(
    bucket, gemini_15_flash, audio_description_extraction_prompt, time_sleep=1
)

In [ ]:
batch_size = 3
max_concurrent = 4

gcs_uri_list_audio = get_gcs_uri_list(bucket, PRODUCTION_DATA, ".mp3")
final_response_list_audio = await batch_and_profile(
    gcs_uri_list_audio,
    audio_description_extraction_prompt,
    gemini_15_pro,  # gemini_15_flash #Switch to flash model for much faster processing or if you are getting any error
    "audio/mpeg",
    batch_size,
    max_concurrent,
)

You may see "Something failed, retrying 429 Resource exhausted...." warning/error while running the above code. You can ignore them and if the execution failed, you can switch to "gemini_15_flash"

In [ ]:
audio_metadata_flash = pd.DataFrame([gcs_uri_list_audio, final_response_list_audio]).T
audio_metadata_flash.columns = ["audio_gcs", "audio_description"]
audio_metadata_flash.head(2)

In [ ]:
print("total files: ....", len(audio_metadata_flash["audio_gcs"].value_counts().index))

In [ ]:
rich_Markdown(audio_metadata_flash["audio_description"][2])

### Video Summary Extraction

![](https://storage.googleapis.com/gemini-lavi-asset/img/Step1_Video.png)

In [ ]:
batch_size = 5
max_concurrent = 4
video_description_extraction_prompt = """Transcribe and analyze the video, intelligently segmenting it based on shifts in topic, focus, or narrative progression.
For each identified segment:
**Concise Summary**: Distill the core theme or message in 1-2 sentences.
**Thematic Context**: How does this segment contribute to the overarching narrative or argument?
**Critical Analysis**: Delve into the segment's implications, perspectives presented, and potential biases.
**Connections**: Link this segment to other parts of the video, revealing patterns and relationships.

Conclude by synthesizing the video's main themes, their interconnections, and the overarching purpose or message.
"""
gcs_uri_list_video = get_gcs_uri_list(bucket, PRODUCTION_DATA, ".mp4")

final_response_list_video = await batch_and_profile(
    gcs_uri_list_video,
    video_description_extraction_prompt,
    gemini_15_pro,  # gemini_15_flash #Switch to flash model for much faster processing or if you are getting any error
    "video/mp4",
    batch_size,
    max_concurrent,
)

You may see "Something failed, retrying 429 Resource exhausted...." warning/error while running the above code. You can ignore them and if the execution failed, you can switch to "gemini_15_flash"

In [ ]:
video_metadata_flash = pd.DataFrame([gcs_uri_list_video, final_response_list_video]).T
video_metadata_flash.columns = ["video_gcs", "video_description"]
video_metadata_flash.head(2)

In [ ]:
print("total files: ....", len(video_metadata_flash["video_gcs"].value_counts().index))

In [ ]:
rich_Markdown(video_metadata_flash["video_description"][3])

### Save the intermediate Files

In [ ]:
# # [Optional]
# import pickle

# data_to_dump = [extracted_text, audio_metadata_flash, video_metadata_flash]

# gcs_location = "gs://mlops-for-genai/multimodal-finanace-qa/data/structured/data_extraction_dataframe.pkl"

# with open("data_extraction_dataframe.pkl", "wb") as f:
#     pickle.dump(data_to_dump, f)


# # Upload the pickle file to GCS
# !gsutil cp data_extraction_dataframe.pkl {gcs_location}